In [1]:
import pandas as pd
import numpy as np

data = pd.read_excel('../data/master.xlsx')
# data = pd.read_excel('../data/master.xlsx').sample(1000)
# data = {'A': [3, 3, 2], 'B': [3, 3, 4], 'C': [7, 7, 7]}
# data = pd.DataFrame(data)

minSup = 1000
numDims = len(data.columns)
outputRec = pd.DataFrame({
    "Combination": [],
    "Count": []
})


In [2]:
data.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')

In [3]:
data.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers


In [4]:
unique_counts = data.nunique()
col_dict = dict(zip(unique_counts.index, unique_counts.values))
sorted_cols = sorted(col_dict, key=col_dict.get, reverse=True)
data = data[sorted_cols]

In [5]:
data.head()

,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),suicides_no,HDI for year,country,year,age,generation,sex
0,312900,6.71,Albania1987,2156624900,796,21,NaN,Albania,1987,15-24 years,Generation X,male
1,308000,5.19,Albania1987,2156624900,796,16,NaN,Albania,1987,35-54 years,Silent,male
2,289700,4.83,Albania1987,2156624900,796,14,NaN,Albania,1987,15-24 years,Generation X,female
3,21800,4.59,Albania1987,2156624900,796,1,NaN,Albania,1987,75+ years,G.I. Generation,male
4,274300,3.28,Albania1987,2156624900,796,9,NaN,Albania,1987,25-34 years,Boomers,male


In [6]:
def Partition(input_df, dim, start_row, end_row, dataCount):
    subset_df = input_df.iloc[start_row:end_row + 1, dim:]
    sorted_df = subset_df.sort_values(by=list(subset_df.columns))
    input_df.iloc[start_row:end_row + 1, dim:] = sorted_df.values
    counts = sorted_df.iloc[:, 0].value_counts().sort_index().tolist()
    dataCount.extend(counts)

In [7]:
def BottomUpCube(input, dim, start_row, end_row, currentCombination, numDims, minSup):
    global outputRec
    for d in range(dim, numDims):
        dataCount = []
        Partition(input, d, start_row, end_row, dataCount)

        k = start_row
        for c in dataCount:
            if c >= minSup:
                outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{
                    "Combination": currentCombination + [input.iloc[k, d]], 
                    "Count": c
                    }])], ignore_index=True)

                BottomUpCube(input, d + 1, k, k + c - 1, currentCombination + [input.iloc[k, d]], numDims, minSup)

            k += c    

In [8]:
count = len(data)
if count >= minSup:
    outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{"Combination": [], "Count": count}])], ignore_index=True)

In [9]:
BottomUpCube(data, 0, 0, len(data) - 1, [], numDims, minSup)

In [10]:
print(outputRec)

             Combination    Count
0                     []  27820.0
1                  [0.0]   4281.0
2               [0.0, 0]   4281.0
3   [0.0, 0, 5-14 years]   1468.0
4   [0.0, 0, Millenials]   1128.0
..                   ...      ...
77    [Millenials, male]   5502.0
78              [Silent]   6364.0
79        [Silent, male]   6364.0
80              [female]  13910.0
81                [male]  13910.0

[82 rows x 2 columns]
